In [2]:
# 优化参数
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm
import random
import pandas as pd


"""加载数据集"""

train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

"""检测设备"""
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"


"""组建神经网络"""
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )


    def forward(self, x):
        x = self.flatten(x)
        logits = self.stack(x)
        return logits

"""将模型转移到设备上"""
model = NeuralNetwork().to(device)

# model.load_state_dict(torch.load('PyTorch_code_7_pth/model_6.pth', weights_only=True))

"""生成迭代器DataLoader"""
# 批次大小
batch_size = 60

train_DataLoader = DataLoader(dataset=train_data, batch_size=batch_size)
test_DataLoader = DataLoader(dataset=test_data, batch_size=batch_size)

"""定义损失函数和优化器"""
loss_def = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.0001)

# 训练轮数
epochs = 300

# 记录每一轮验证集上的损失函数值和正确率。
Epoch = [index for index in range(1, epochs + 1)]
Train_Loss = []
Test_Loss = []
Correctness = []

# 封装训练函数
def train(model, dataloader, loss_def, optim):
    data_size = len(dataloader.dataset)
    iter_size, batch_train_loss= len(dataloader), 0
    model.train()
    count = 0
    for Batch, (Images, Labels) in enumerate(dataloader):
        Images, Labels = Images.to(device), Labels.to(device)
        logits = model(Images)
        loss = loss_def(logits, Labels)
        loss.backward()
        count += 1
        if count % 4 == 0:
            optim.step()
            optim.zero_grad()
        if (Batch + 1) % 100 == 0:
            loss, current = loss.item(), (Batch + 1) * len(Images)
            print(f"Loss: {loss:>6f}   |   [current: {current:>5d}/{data_size}]")
        batch_train_loss += loss
    train_loss = batch_train_loss / iter_size
    return train_loss


# 封装验证函数
def test(model, dataloader, loss_def):
    data_size = len(dataloader.dataset)
    batchs = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for batch, (Image, Label) in enumerate(dataloader):
            Image, Label = Image.to(device), Label.to(device)
            logits = model(Image)
            test_loss += loss_def(logits, Label)
            correct += (logits.argmax(1) == Label).type(torch.float).sum().item()
    result = correct / data_size
    batch_loss = test_loss / batchs
    print(f"Test Error:\ntest_correct: {result:>.6f}   |   [test_loss: {batch_loss:>.6f}]")
    return result, batch_loss


# 进行循环多轮训练
for index in range(epochs):
    print(f"Epochs {index + 1}" + "\n+" + "-" * 80)
    train_loss = train(model, train_DataLoader, loss_def, optim)
    result, batch_loss = test(model, test_DataLoader, loss_def)
    Train_Loss.append(train_loss.item())
    Correctness.append(result)
    Test_Loss.append(batch_loss.item())


# 保存模型训练参数
path = "PyTorch_code_7_pth/model_8.pth"
torch.save(model.state_dict(), path)
print(f"模型参数已保存到 '{path}'")


# 保存csv文件
data = {
    'Epoch': Epoch,
    'Train_Loss': Train_Loss,
    'Test_Loss': Test_Loss,
    'Correctness': Correctness
}

df = pd.DataFrame(data)
out_path = "/home/SayMyName/桌面/GitHub/MyCode/DeepLearn/PyTorch/PyTorch_code_7_CSV/PyTorch_code_7_3.csv"
df.to_csv(out_path, index=False)
print(f"CSV文件已经保存到 {out_path}")

# 加载模型参数
# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("PyTorch_code_7_pth/model_1.pth", weights_only=True))
#
#
# # 进行预测
# classes = [
#     "T-shirt/top",
#     "Trouser",
#     "Pullover",
#     "Dress",
#     "Coat",
#     "Sandal",
#     "Shirt",
#     "Sneaker",
#     "Bag",
#     "Ankle boot",
# ]
#
# rand = random.randint(0, 1000)
#
# model.eval()
# Image, Label = test_data[rand][0], test_data[rand][1]
# with torch.no_grad():
#     Image = Image.to(device)
#     predict = model(Image)
#     print(predict)
#     print(predict[0].argmax(0))
#     predicted, actual = classes[predict[0].argmax(0)], classes[Label]
#
# print(f"Predicted: {predicted}   |   Actual: {actual}")




Epochs 1
+--------------------------------------------------------------------------------
Loss: 2.304946   |   [current:  6000/60000]
Loss: 2.298654   |   [current: 12000/60000]
Loss: 2.296955   |   [current: 18000/60000]
Loss: 2.313061   |   [current: 24000/60000]
Loss: 2.289837   |   [current: 30000/60000]
Loss: 2.292866   |   [current: 36000/60000]
Loss: 2.284511   |   [current: 42000/60000]
Loss: 2.296060   |   [current: 48000/60000]
Loss: 2.288451   |   [current: 54000/60000]
Loss: 2.291455   |   [current: 60000/60000]
Test Error:
test_correct: 0.100100   |   [test_loss: 2.288453]
Epochs 2
+--------------------------------------------------------------------------------
Loss: 2.292328   |   [current:  6000/60000]
Loss: 2.286754   |   [current: 12000/60000]
Loss: 2.284161   |   [current: 18000/60000]
Loss: 2.305609   |   [current: 24000/60000]
Loss: 2.275342   |   [current: 30000/60000]
Loss: 2.279373   |   [current: 36000/60000]
Loss: 2.270749   |   [current: 42000/60000]
Loss: 2